In [1]:
%load_ext autoreload
%load_ext autotime
%autoreload 2

time: 1.55 ms (started: 2021-11-05 22:02:20 -07:00)


In [2]:
import requests, math, sys, os
from bs4 import BeautifulSoup
import spacy
from spacy import displacy
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import time

#  setting path
gparent = os.path.join(os.pardir)
sys.path.append(gparent)

from src import helper_functions as h

time: 4.64 s (started: 2021-11-05 22:02:20 -07:00)


## requests: Grabbing Headlines

In [3]:
# grabbing headlines
r = requests.get("http://feeds.marketwatch.com/marketwatch/topstories/")
r

<Response [200]>

time: 452 ms (started: 2021-11-05 22:02:25 -07:00)


## bs4: Saving Headlines to a List

In [4]:
# saving headlines to a list
soup = BeautifulSoup(r.content, features='lxml')
headlines = soup.findAll('title')
headlines

[<title>MarketWatch.com - Top Stories</title>,
 <title>MarketWatch.com - Top Stories</title>,
 <title>: House approves infrastructure bill, sending it to Biden to be signed into law</title>,
 <title>The Moneyist: My wife made her sister 401(k) beneficiary. I asked to be listed in case she dies first — she promised her sister would ‘give me the money.’ What should I do?</title>,
 <title>The Moneyist: My husband bought a home with his brother for $350K, but stopped paying the mortgage in 2012. Now it’s worth $700K. How should they split the proceeds?</title>,
 <title>The Big Move: I own a rental home in California. How can I leave it to my daughter so she can avoid capital gains taxes?</title>,
 <title>Market Extra: Corporate America’s borrowing binge seen barely slowing as Federal Reserve tapers</title>,
 <title>The Wall Street Journal: Attorneys general from 11 states sue Biden administration over vaccine mandate</title>,
 <title>MemeMoney: AMC’s Adam Aron has a deep and abiding belief

time: 11 ms (started: 2021-11-05 22:02:26 -07:00)


### Grabbing Headline String
Grabbing headline and printing headline and isolated text.

In [5]:
# grabbing test headline
test_headline = headlines[2]
print(test_headline)
print(test_headline.text)

<title>: House approves infrastructure bill, sending it to Biden to be signed into law</title>
: House approves infrastructure bill, sending it to Biden to be signed into law
time: 1.79 ms (started: 2021-11-05 22:02:26 -07:00)


## spaCy: Tests
Testing tokenization and name extraction on a single string.

### Loading Model

In [6]:
# loading spacy model
nlp = spacy.load('en_core_web_sm')

time: 883 ms (started: 2021-11-05 22:02:26 -07:00)


### Tokenizing String
Tokenizing the test headline.

In [7]:
# checking the test case
processed_hline = nlp(test_headline.text)
print(test_headline)
for token in processed_hline:
  print(token)

<title>: House approves infrastructure bill, sending it to Biden to be signed into law</title>
:
House
approves
infrastructure
bill
,
sending
it
to
Biden
to
be
signed
into
law
time: 18.7 ms (started: 2021-11-05 22:02:27 -07:00)


## Getting Org Names From Headlines
Visualizing named entities (real world objects) in the headlines and creating a set of organiziations.

In [8]:
# pulling company name tokens from headlines
companies = []
for title in headlines:
    doc = nlp(title.text)
    if len(doc.ents) != 0:
        displacy.render(doc, style='ent')
    else:
        pass
    for token in doc.ents:
        if token.label_ == 'ORG':
            companies.append(token.text)
        else:
            pass
companies = set(companies)
print(companies)

{'Federal Reserve', 'The Wall Street Journal', 'House'}
time: 278 ms (started: 2021-11-05 22:02:27 -07:00)


## Scraping S&P 500 Stock Table w/Requests & BeautifulSoup

In [9]:
# scraping S&P wikipedia page
r = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# parsing the html
soup = BeautifulSoup(r.text, 'lxml')

# extracting the table
table = soup.find('table', {'class': 'wikitable sortable'})

# printing row with first stock
print(table.findAll('tr')[1:2])

[<tr>
<td><a class="external text" href="https://www.nyse.com/quote/XNYS:MMM" rel="nofollow">MMM</a>
</td>
<td><a href="/wiki/3M" title="3M">3M</a></td>
<td><a class="external text" href="https://www.sec.gov/edgar/browse/?CIK=66740" rel="nofollow">reports</a></td>
<td>Industrials</td>
<td>Industrial Conglomerates</td>
<td><a href="/wiki/Saint_Paul,_Minnesota" title="Saint Paul, Minnesota">Saint Paul, Minnesota</a></td>
<td>1976-08-09</td>
<td>0000066740</td>
<td>1902
</td></tr>]
time: 832 ms (started: 2021-11-05 22:02:27 -07:00)


### Ticker Symbols
Grabbing the ticker symbol from the first cell of each row.

In [10]:
# making list of symbols
symbols = [row.findAll('td')[0].text for row in table.findAll('tr')[1:]]

# checking first 5 symbols
print(symbols[:5])

['MMM\n', 'ABT\n', 'ABBV\n', 'ABMD\n', 'ACN\n']
time: 25.4 ms (started: 2021-11-05 22:02:28 -07:00)


In [11]:
# stripping new line character from the strings 
symbols = list(map(lambda s: s.strip(), symbols))

# checking first 5 symbols
print(symbols[:5])

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN']
time: 1.51 ms (started: 2021-11-05 22:02:28 -07:00)


### Names
Grabbing the company name from the second cell of each row.

In [12]:
# making list of names
names = [row.findAll('td')[1].text for row in table.findAll('tr')[1:]]

# checking first five names
print(names[:5])

['3M', 'Abbott Laboratories', 'AbbVie', 'Abiomed', 'Accenture']
time: 22.1 ms (started: 2021-11-05 22:02:28 -07:00)


### Industries
Grabbing the company industry from the fifth cell of each row.

In [13]:
# making list of industries
industries = [row.findAll('td')[4].text for row in table.findAll('tr')[1:]]

# checking first five names
print(industries[:5])

['Industrial Conglomerates', 'Health Care Equipment', 'Pharmaceuticals', 'Health Care Equipment', 'IT Consulting & Other Services']
time: 24.4 ms (started: 2021-11-05 22:02:28 -07:00)


## S&P 500 Dataframe
Creating a data frame of stocks in the S&P 500 index.

In [14]:
# making a data dictionary
data = {'Company Name': names, 'Symbol': symbols, 'Industry': industries}

# creating data frame from the data
stocks_df = pd.DataFrame.from_dict(data)

# checking shape and first five rows
print(stocks_df.shape)
stocks_df.head()

(505, 3)


,Company Name,Symbol,Industry
0,3M,MMM,Industrial Conglomerates
1,Abbott Laboratories,ABT,Health Care Equipment
2,AbbVie,ABBV,Pharmaceuticals
3,Abiomed,ABMD,Health Care Equipment
4,Accenture,ACN,IT Consulting & Other Services


time: 19.4 ms (started: 2021-11-05 22:02:28 -07:00)


### Checking yf Stock Info Dictionary Keys
Checking the dictionary keys available for pulling stock information.

In [15]:
# instantiating a ticker object
ACN = yf.Ticker('ACN')

time: 5.92 ms (started: 2021-11-05 22:02:29 -07:00)


In [16]:
# checking keys
ACN.info.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'fax', 'industry', 'address2', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskR

time: 3.03 s (started: 2021-11-05 22:02:29 -07:00)


### Data for Stocks in the News
Creating a data frame of price and dividend information for S&P 500 stocks in the news in the following steps:

- Create a data dictionary from the list of companies in the headlines.

- Create a data frame from data dictionary.

In [17]:
# creating empty stock info dictionary
stock_data = {
    'Company': [],
    'Symbol': [],
    'currentPrice': [],
    'dayHigh': [],
    'dayLow': [],
    '52wkHigh': [],
    '52wkLow': [],
    'dividendRate': []
    
}

# loading stocks from s&p dataframe and appending data from yf
for company in companies:

    try:
        if stocks_df['Company Name'].str.contains(company).sum():
            symbol = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Symbol'].values[0]
            org_name = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Company Name'].values[0]
            stock_data['Company'].append(org_name)
            stock_data['Symbol'].append(symbol)
            stock_info = yf.Ticker(symbol).info
            stock_data['currentPrice'].append(stock_info['currentPrice'])
            stock_data['dayHigh'].append(stock_info['dayHigh'])
            stock_data['dayLow'].append(stock_info['dayLow'])
            stock_data['52wkHigh'].append(stock_info['fiftyTwoWeekHigh'])
            stock_data['52wkLow'].append(stock_info['fiftyTwoWeekLow'])            
            
            # converting dividend None types to floats
            dividend = stock_info['dividendRate']
            if dividend != None:
                dividend = dividend
            else:
                dividend = 0
            stock_data['dividendRate'].append(dividend)
        else:
            pass
    except:
        pass

time: 5.18 ms (started: 2021-11-05 22:02:32 -07:00)


In [18]:
# checking dict
stock_data

{'Company': [],
 'Symbol': [],
 'currentPrice': [],
 'dayHigh': [],
 'dayLow': [],
 '52wkHigh': [],
 '52wkLow': [],
 'dividendRate': []}

time: 2.76 ms (started: 2021-11-05 22:02:32 -07:00)


### Data Frame of S&P 500 Stocks in the News

In [19]:
in_the_news = pd.DataFrame(stock_data)
in_the_news.head()

,Company,Symbol,currentPrice,dayHigh,dayLow,52wkHigh,52wkLow,dividendRate


time: 6.99 ms (started: 2021-11-05 22:02:32 -07:00)


In [20]:
# checking Dtypes
in_the_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company       0 non-null      float64
 1   Symbol        0 non-null      float64
 2   currentPrice  0 non-null      float64
 3   dayHigh       0 non-null      float64
 4   dayLow        0 non-null      float64
 5   52wkHigh      0 non-null      float64
 6   52wkLow       0 non-null      float64
 7   dividendRate  0 non-null      float64
dtypes: float64(8)
memory usage: 124.0 bytes
time: 6.17 ms (started: 2021-11-05 22:02:32 -07:00)


## Individual Stock Price Helper Function
Checking the helper function.

In [23]:
h.prices('MMM')

,Company,Symbol,currentPrice,dayHigh,dayLow,52wkHigh,52wkLow,dividendRate
0,3M Company,MMM,181.8,183.82,180.95,208.95,163.15,5.92


time: 3.12 s (started: 2021-11-05 22:05:41 -07:00)


In [24]:
h.prices('AAPL')

,Company,Symbol,currentPrice,dayHigh,dayLow,52wkHigh,52wkLow,dividendRate
0,Apple Inc.,AAPL,151.28,152.1899,150.06,157.26,112.59,0.88


time: 2.88 s (started: 2021-11-05 22:05:45 -07:00)
